In [1]:
import torch
import torch.nn as nn
import onnx
# import tdqm
# import torch.nn.functional as F
# from torch.nn import Parameter

/net/jabba/home0/am611608/source/simple-hpc/venv/lib64/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dummy model

In [2]:
class ToyNN(nn.Module):
    def __init__(self):
        super().__init__()
        input_dim = 8
        self.l1 = nn.Linear(input_dim, input_dim//2)
        self.l2 = nn.Linear(input_dim//2, input_dim//4)
        self.l3 = nn.Linear(input_dim//4, input_dim//8)
        self.activ = nn.Sigmoid()

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return self.activ(x)

In [3]:
def get_n_params(model):
    """ Returns the number of learnable parameters of a network
    """
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [4]:
toy = ToyNN()
print(f"Number of parameters : {get_n_params(toy)}")
print(toy)

Number of parameters : 49
ToyNN(
  (l1): Linear(in_features=8, out_features=4, bias=True)
  (l2): Linear(in_features=4, out_features=2, bias=True)
  (l3): Linear(in_features=2, out_features=1, bias=True)
  (activ): Sigmoid()
)


In [7]:
for i, layer in enumerate(toy.children()):
    if isinstance(layer, nn.Linear):
        print(f"Layer {i+1} : ")
        print("\t Weight :\n", layer.state_dict()['weight'])
        print("\t Bias :\n", layer.state_dict()['bias'])

Layer 1 : 
	 Weight :
 tensor([[-0.0094,  0.1759, -0.2755,  0.2614,  0.2872,  0.1945, -0.1402,  0.3175],
        [-0.2218, -0.0506,  0.1850,  0.0422,  0.0371,  0.1627, -0.1654, -0.2201],
        [ 0.1564, -0.0031, -0.0359, -0.1669,  0.1859,  0.3320, -0.2776, -0.0548],
        [-0.0659, -0.1149, -0.2432, -0.2087, -0.1644,  0.2388,  0.0289, -0.1584]])
	 Bias :
 tensor([-0.3439, -0.2593,  0.2180,  0.2058])
Layer 2 : 
	 Weight :
 tensor([[ 0.0025,  0.0133,  0.2799, -0.0356],
        [-0.1392, -0.2166, -0.3421,  0.2777]])
	 Bias :
 tensor([-0.2735,  0.4990])
Layer 3 : 
	 Weight :
 tensor([[0.3379, 0.1290]])
	 Bias :
 tensor([-0.6833])


# ONNX Export

In [6]:
toy.eval()

ToyNN(
  (l1): Linear(in_features=8, out_features=4, bias=True)
  (l2): Linear(in_features=4, out_features=2, bias=True)
  (l3): Linear(in_features=2, out_features=1, bias=True)
  (activ): Sigmoid()
)

In [12]:
x = torch.randn(1, 8)
out = toy(x)

torch.onnx.export(
    toy,
    x,
    "toy-model.onnx",
    export_params=True,
    opset_version=10,
    # input_names=['input'],
    # output_names=['output']
)


In [13]:
import onnx

onnx_model = onnx.load("toy-model.onnx")
onnx.checker.check_model(onnx_model)


In [10]:


# input_example = torch.Tensor([1.0,1.0,1.0,1.0,1.,2.,1.,1.])

# torch.onnx.export(
#   toy,
#   input_example,
#   "toy-model.onnx",
#   # only needed if you want to specify an opset version > 1.
#   custom_opsets={"custom_domain": 2})

# Dummy training

In [16]:
optimizer = torch.optim.Adam(toy.parameters(), lr = 0.0001)

def criterion(x, y):
    return (x - y)**2

def train():
    # toy.zero_grad()
    input_data = torch.randn((8,))

    pred = toy(input_data)
    loss = criterion(pred, 0.7)
    loss.backward()
    optimizer.step()
    return loss.item(), pred.item()

In [17]:
epochs = range(1000)
for e in epochs :
    print(f"{e} : {train()}")


0 : (0.023232106119394302, 0.5475791692733765)
1 : (0.023674266412854195, 0.5461355447769165)
2 : (0.023647183552384377, 0.5462235808372498)
3 : (0.023774612694978714, 0.545809805393219)
4 : (0.0240115225315094, 0.5450434684753418)
5 : (0.0241505429148674, 0.5445955395698547)
6 : (0.023206965997815132, 0.5476616621017456)
7 : (0.02361918054521084, 0.5463146567344666)
8 : (0.02364569902420044, 0.5462284088134766)
9 : (0.02302340418100357, 0.5482653379440308)
10 : (0.023720217868685722, 0.5459862947463989)
11 : (0.02337649092078209, 0.5471062660217285)
12 : (0.023457102477550507, 0.5468428730964661)
13 : (0.02373022399842739, 0.5459538102149963)
14 : (0.023158449679613113, 0.5478209853172302)
15 : (0.02354273572564125, 0.5465635657310486)
16 : (0.02391897328197956, 0.5453423857688904)
17 : (0.023637762293219566, 0.5462542176246643)
18 : (0.0230956319719553, 0.548027515411377)
19 : (0.022863691672682762, 0.5487925410270691)
20 : (0.023561066016554832, 0.5465038418769836)
21 : (0.023289231

In [18]:
for i, layer in enumerate(toy.children()):
    if isinstance(layer, nn.Linear):
        print(f"Layer {i+1} : ")
        print("\t Weight :\n", layer.state_dict()['weight'])
        print("\t Bias :\n", layer.state_dict()['bias'])

Layer 1 : 
	 Weight :
 tensor([[-0.2309, -0.1180,  0.2913,  0.1234, -0.0551,  0.0950, -0.0720, -0.0351],
        [-0.1571,  0.0459,  0.2048, -0.1470,  0.4072,  0.3345,  0.2768, -0.3372],
        [ 0.2265,  0.0122, -0.2130,  0.4028, -0.2867, -0.1345,  0.1785,  0.1515],
        [-0.2350, -0.1964, -0.2270, -0.1283,  0.1948,  0.1162,  0.1746,  0.0491]])
	 Bias :
 tensor([ 0.3948, -0.4184,  0.3052,  0.4409])
Layer 2 : 
	 Weight :
 tensor([[ 0.0563, -0.5949,  0.4556,  0.6765],
        [ 0.1586, -0.0319,  0.6026,  0.0536]])
	 Bias :
 tensor([0.3135, 0.3934])
Layer 3 : 
	 Weight :
 tensor([[0.4032, 0.7504]])
	 Bias :
 tensor([0.4877])
Layer 4 : 
	 Weight :
 tensor([[0.1753]])
	 Bias :
 tensor([0.3277])
